In [1]:
# load libraries
import os
import xml.etree.cElementTree as cET
from collections import defaultdict
import pprint
import re
import codecs
import json
import string
from pymongo import MongoClient

In [16]:
# set up map file path
# filename = "MountainView.osm" # osm filename
filename = "sample200.osm" # Sample osm filename
path = "/Users/seemamishra/Desktop/Udacity/Data analyst nano degree/Data Wrangling/P3_Data" # directory contain the osm file
MountainViewosm = os.path.join(path, filename)

# MountainViewosm = "MountainView.osm" # osm filename
# path = "d:\GithubRepos\Udacity\P3" # directory contain the osm file
lower = re.compile(r'^([a-z]|_)*$') 
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)
# initial version of expected street names
expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane",
            "Road", "Trail", "Parkway", "Commons"]
MountainViewosm

'/Users/seemamishra/Desktop/Udacity/Data analyst nano degree/Data Wrangling/P3_Data/sample200.osm'

In [17]:
# Iterative parsing
def count_tags(filename):
    
    # make empty defaultdict
#     from collections import defaultdict
    tags_dict = defaultdict(int)
    
    # use the iterparse method to find all the tags
    for event, element in cET.iterparse(filename, events=("start", "end")):
#         print event
        tags_dict[element.tag] += 1
        
    # return your results 
    return tags_dict

if __name__ == "__main__":
    print count_tags(MountainViewosm)

defaultdict(<type 'int'>, {'node': 10242, 'nd': 11918, 'member': 34, 'tag': 4054, 'relation': 12, 'way': 1234, 'osm': 2})


In [18]:
# Tag types
def key_type(element, keys):
    if element.tag == "tag":
    
        k = element.attrib['k']
#         print k
        # serach k to see if it matches each regular expression
        if lower.search(k):
            keys['lower'] += 1
        elif lower_colon.search(k):
            keys['lower_colon'] += 1
        elif problemchars.search(k):
            keys['problemchars'] += 1
        else:
            keys['other'] += 1
           
    return keys



def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in cET.iterparse(filename):
        keys = key_type(element, keys)

    return keys


if __name__ == "__main__":
    print process_map(MountainViewosm)

{'problemchars': 0, 'lower': 1120, 'other': 26, 'lower_colon': 881}


In [21]:
def audit_street_type(street_types, street_name):
    # add unexpected street name to a list
    m = street_type_re.search(street_name)
#     print m
    if m:
        street_type = m.group()
#         street_type
        if street_type not in expected:
            street_types[street_type].add(street_name)
            
def is_street_name(elem):
    # determine whether a element is a street name
    return (elem.attrib['k'] == "addr:street")

def audit_street(osmfile):
    # iter through all street name tag under node or way and audit the street name value
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in cET.iterparse(osm_file, events=("start","end")):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    return street_types
if __name__ == '__main__':
    st_types = audit_street(MountainViewosm)
    # print out unexpected street names
    pprint.pprint(dict(st_types))




{'Circle': set(['Redwood Circle']),
 'Real': set(['West El Camino Real']),
 'Ridge': set(['Roble Ridge']),
 'Terrace': set(['Avoset Terrace', 'Springer Terrace']),
 'Walk': set(['Altaire Walk']),
 'Way': set(['Anaconda Way',
             'Dartshire Way',
             'Durshire Way',
             'Edmonds Way',
             'Enderby Way',
             'Fabian Way',
             'Golden Way',
             'Janice Way',
             'La Jennifer Way',
             'Mallard Way',
             'Nelson Way',
             'Saint Julien Way',
             'Tartarian Way',
             'Thain Way',
             'Towle Way'])}


In [22]:
# Street name updatation
# creating a dictionary for correcting street names
mapping = { "AA" :"Aberdeen Athletic Center",
            "Ct": "Court",
            "Ct.": "Court",
            "St": "Street",
            "st": "Street",
            "St.": "Street",
            "St,": "Street",
            "ST": "Street",
            "street": "Street",
            "Street.": "Street",
            "STE": "Street",
            "Ave": "Avenue",
            "Ave.": "Avenue",
            "ave": "Avenue",
            "Rd.": "Road",   
            "rd.": "Road",
            "Rd": "Road",    
            "Hwy": "Highway",
            "HIghway": "Highway",
            "BLDG": "Building",
            "APT": "Apartment",
           "West Evelyn Avenue Suite #114":"West Evelyn Avenue",
           "Showers Drive STE 2": "Showers Drive Street",
           "Showers Drive STE 7": "Showers Drive Street",
           "East Charleston Road APT 9": "East Charleston Road Apartment",
           "Leghorn Street #B": "Leghorn Street",
           "Plymouth Street #C": "Plymouth Street",
           "Hamilton Ave #140": "Hamilton Ave"
           }
           
                     
# function that corrects incorrect street names
def update_name(name, mapping):    
    for key in mapping:
        if key in name:
            name = string.replace(name,key,mapping[key])
    return name
if __name__ == '__main__':
    for st_type, ways in st_types.iteritems():
        for name in ways:
            better_name = update_name(name, mapping)
            print name, "=>", better_name

West El Camino Real => WeStreet El Camino Real
Roble Ridge => Roble Ridge
Altaire Walk => Altaire Walk
Springer Terrace => Springer Terrace
Avoset Terrace => Avoset Terrace
Towle Way => Towle Way
Edmonds Way => Edmonds Way
Tartarian Way => Tartarian Way
Thain Way => Thain Way
Durshire Way => Durshire Way
Nelson Way => Nelson Way
Fabian Way => Fabian Way
Mallard Way => Mallard Way
La Jennifer Way => La Jennifer Way
Janice Way => Janice Way
Anaconda Way => Anaconda Way
Saint Julien Way => Saint Julien Way
Golden Way => Golden Way
Enderby Way => Enderby Way
Dartshire Way => Dartshire Way
Redwood Circle => Redwood Circle


In [23]:
def audit_zipcodes(osmfile):
    # iter through all zip codes, collect all the zip codes that does not start with 02
    osm_file = open(osmfile, "r")
    zip_codes = {}
    for event, elem in cET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if tag.attrib['k'] == "addr:postcode" and not tag.attrib['v'].startswith('02'):
                    if tag.attrib['v'] not in zip_codes:
                        zip_codes[tag.attrib['v']] = 1
                    else:
                        zip_codes[tag.attrib['v']] += 1
    return zip_codes

zipcodes = audit_zipcodes(MountainViewosm)
for zipcode in zipcodes:
    print zipcode, zipcodes[zipcode]

95014-0529 1
94305 1
94043 9


In [24]:
# Process OSM XML file to JSON
CREATED = [ "version", "changeset", "timestamp", "user", "uid"]
def shape_element(element):
    node = {}
    node["created"]={}
    node["address"]={}
    node["pos"]=[]
    node["amenity"] ={}
    node["cuisine"] = {}
    refs=[]
    
    # we only process the node and way tags
    if element.tag == "node" or element.tag == "way" :
        if "id" in element.attrib:
            node["id"]=element.attrib["id"]
        node["type"]=element.tag

        if "visible" in element.attrib.keys():
            node["visible"]=element.attrib["visible"]
      
        # the key-value pairs with attributes in the CREATED list are added under key "created"
        for elem in CREATED:
            if elem in element.attrib:
                node["created"][elem]=element.attrib[elem]
                
        # attributes for latitude and longitude are added to a "pos" array
        # include latitude value        
        if "lat" in element.attrib:
            node["pos"].append(float(element.attrib["lat"]))
        # include longitude value    
        if "lon" in element.attrib:
            node["pos"].append(float(element.attrib["lon"]))

        
        for tag in element.iter("tag"):
            if not(problemchars.search(tag.attrib['k'])):
                if tag.attrib['k'] == "addr:housenumber":
                    node["address"]["housenumber"]=tag.attrib['v']
                    
                if tag.attrib['k'] == "addr:postcode":
                    node["address"]["postcode"]=tag.attrib['v']
                
                # handling the street attribute, update incorrect names using the strategy developed before   
                if tag.attrib['k'] == "addr:street":
                    node["address"]["street"]=tag.attrib['v']
                    node["address"]["street"] = update_name(node["address"]["street"], mapping)

                if tag.attrib['k'].find("addr")==-1:
                    node[tag.attrib['k']]=tag.attrib['v']
                    
        for nd in element.iter("nd"):
             refs.append(nd.attrib["ref"])
                
        if node["address"] =={}:
            node.pop("address", None)

        if refs != []:
           node["node_refs"]=refs
            
        return node
    else:
        return None



# process the xml openstreetmap file, write a json out file and return a list of dictionaries
def process_map(file_in, pretty = False):
    file_out = "{0}.json".format(file_in)
    data = []
    with codecs.open(file_out, "w") as fo:
        for _, element in cET.iterparse(file_in):
            el = shape_element(element)
            if el:
                data.append(el)
                if pretty:
                    fo.write(json.dumps(el, indent=2)+"\n")
                else:
                    fo.write(json.dumps(el) + "\n")
    return data

In [25]:
# process the file
data = process_map(MountainViewosm, True)


In [16]:
data

[{'created': {'changeset': '6776161',
   'timestamp': '2010-12-27T10:41:10Z',
   'uid': '14293',
   'user': 'KindredCoda',
   'version': '4'},
  'id': '302863',
  'pos': [37.3769826, -122.0677726],
  'type': 'node'},
 {'created': {'changeset': '6776161',
   'timestamp': '2010-12-27T10:39:22Z',
   'uid': '14293',
   'user': 'KindredCoda',
   'version': '5'},
  'id': '302864',
  'pos': [37.3740667, -122.0679336],
  'type': 'node'},
 {'created': {'changeset': '6774664',
   'timestamp': '2010-12-27T07:44:11Z',
   'uid': '14293',
   'user': 'KindredCoda',
   'version': '4'},
  'id': '302865',
  'pos': [37.3733004, -122.0677211],
  'type': 'node'},
 {'created': {'changeset': '6774664',
   'timestamp': '2010-12-27T07:43:52Z',
   'uid': '14293',
   'user': 'KindredCoda',
   'version': '4'},
  'id': '302866',
  'pos': [37.372597, -122.0673831],
  'type': 'node'},
 {'created': {'changeset': '6774664',
   'timestamp': '2010-12-27T07:43:25Z',
   'uid': '14293',
   'user': 'KindredCoda',
   'versio

In [30]:
client = MongoClient()
db = client.MountainViewosm
collection = db.MountainViewMAP
collection.insert(data)

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


[ObjectId('58e6bfb31750d204b3ec3829'),
 ObjectId('58e6bfb31750d204b3ec382a'),
 ObjectId('58e6bfb31750d204b3ec382b'),
 ObjectId('58e6bfb31750d204b3ec382c'),
 ObjectId('58e6bfb31750d204b3ec382d'),
 ObjectId('58e6bfb31750d204b3ec382e'),
 ObjectId('58e6bfb31750d204b3ec382f'),
 ObjectId('58e6bfb31750d204b3ec3830'),
 ObjectId('58e6bfb31750d204b3ec3831'),
 ObjectId('58e6bfb31750d204b3ec3832'),
 ObjectId('58e6bfb31750d204b3ec3833'),
 ObjectId('58e6bfb31750d204b3ec3834'),
 ObjectId('58e6bfb31750d204b3ec3835'),
 ObjectId('58e6bfb31750d204b3ec3836'),
 ObjectId('58e6bfb31750d204b3ec3837'),
 ObjectId('58e6bfb31750d204b3ec3838'),
 ObjectId('58e6bfb31750d204b3ec3839'),
 ObjectId('58e6bfb31750d204b3ec383a'),
 ObjectId('58e6bfb31750d204b3ec383b'),
 ObjectId('58e6bfb31750d204b3ec383c'),
 ObjectId('58e6bfb31750d204b3ec383d'),
 ObjectId('58e6bfb31750d204b3ec383e'),
 ObjectId('58e6bfb31750d204b3ec383f'),
 ObjectId('58e6bfb31750d204b3ec3840'),
 ObjectId('58e6bfb31750d204b3ec3841'),
 ObjectId('58e6bfb31750d2

In [18]:
collection

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), u'MountainViewosm'), u'MountainViewMAP')

In [26]:

os.path.getsize(MountainViewosm)/1024/1024

1

In [28]:

os.path.getsize(os.path.join(path, "sample200.osm.json"))/1024/1024

1

In [31]:
collection.find().count()


1153227

In [32]:
# Number of unique users
len(collection.group(["created.uid"], {}, {"count":0}, "function(o, p){p.count++}"))


880

In [33]:
# Number of nodes
collection.find({"type":"node"}).count()

1029309

In [34]:
collection.find({"type":"way"}).count()

123907

In [36]:
# The Number of Methods Used to Create Data Entry
pipeline = [{"$group":{"_id": "$created_by",
                       "count": {"$sum": 1}}}]
result = collection.aggregate(pipeline)
for r in result:
    print r
# assert len(result['result'])

# print(len(result['result']))
# print result[result]

{u'count': 1, u'_id': u'Potlatch 0.8b'}
{u'count': 1, u'_id': u'Potlatch 0.6c'}
{u'count': 20, u'_id': u'Potlatch 0.10e'}
{u'count': 1, u'_id': u'Potlatch 0.10a'}
{u'count': 1, u'_id': u'Potlatch 0.8a'}
{u'count': 21, u'_id': u'Potlatch 0.9a'}
{u'count': 1, u'_id': u'xybot'}
{u'count': 1, u'_id': u'OSMPointy v0.3'}
{u'count': 49, u'_id': u'Potlatch 0.10'}
{u'count': 2, u'_id': u'Merkaartor 0.13'}
{u'count': 875, u'_id': u'JOSM'}
{u'count': 43, u'_id': u'Potlatch 0.8c'}
{u'count': 298, u'_id': u'Potlatch 0.10f'}
{u'count': 3, u'_id': u'iLOE 1.9'}
{u'count': 58, u'_id': u'Potlatch 0.10b'}
{u'count': 16, u'_id': u'OSMPointy v0.4 iPhone'}
{u'count': 2, u'_id': u'Potlatch 0.10d'}
{u'count': 180, u'_id': u'Potlatch 0.9c'}
{u'count': 1151654, u'_id': None}


In [37]:
# top three users with most contributions
pipeline = [{"$group":{"_id": "$created.user",
                       "count": {"$sum": 1}}},
            {"$sort": {"count": -1}},
            {"$limit": 3}]
result = collection.aggregate(pipeline)
for r in result:
    print r

{u'count': 152400, u'_id': u'RichRico'}
{u'count': 141765, u'_id': u'ediyes'}
{u'count': 120485, u'_id': u'samely'}


In [38]:
# Most popular cuisines
pipeline = [{"$match":{"amenity":{"$exists":1}, "amenity":"restaurant", "cuisine":{"$exists":1}}}, 
            {"$group":{"_id":"$cuisine", "count":{"$sum":1}}},        
            {"$sort":{"count":-1}}, 
            {"$limit":10}]
result = collection.aggregate(pipeline)
for r in result:
    print r

{u'count': 38, u'_id': u'mexican'}
{u'count': 35, u'_id': u'chinese'}
{u'count': 34, u'_id': u'japanese'}
{u'count': 29, u'_id': u'indian'}
{u'count': 26, u'_id': u'pizza'}
{u'count': 16, u'_id': u'thai'}
{u'count': 15, u'_id': u'italian'}
{u'count': 13, u'_id': u'american'}
{u'count': 13, u'_id': u'vietnamese'}
{u'count': 12, u'_id': u'burger'}


In [39]:
# University
pipeline = [{"$match":{"amenity":{"$exists":1}, "amenity": "university", "name":{"$exists":1}}},
            {"$group":{"_id":"$name", "count":{"$sum":1}}},
            {"$sort":{"count":-1}}]
result = collection.aggregate(pipeline)
for r in result:
    print r


{u'count': 1, u'_id': u'Stanford University'}
{u'count': 1, u'_id': u'Carnegie Mellon University Silicon Valley'}
{u'count': 1, u'_id': u'Singularity University Classroom '}
{u'count': 1, u'_id': u'Singularity University'}
{u'count': 1, u'_id': u'20'}
{u'count': 1, u'_id': u'Nine Star University of Health Sciences'}


In [41]:
pipeline = [ {"$match":{"amenity":"place_of_worship","type":"node"}}, {"$group":{"_id":"$religion","count":{"$sum":1}}}, {"$sort":{"count":-1}}, {"$limit":1} ]) [ { "_id" : "christian", "count" : 125 } ]

SyntaxError: invalid syntax (<ipython-input-41-f3e39a4c223f>, line 1)

In [43]:
pipeline = [{"$match":{"amenity":{"$exists":1}, "amenity": "place_of_worship", "name":{"$exists":1}}},
            {"$group":{"_id":"$name", "count":{"$sum":1}}},
            {"$sort":{"count":-1}}]
result = collection.aggregate(pipeline)
for r in result:
    print r

{u'count': 2, u'_id': u'The Church of Jesus Christ of Latter-day Saints'}
{u'count': 2, u'_id': u'First Church of Christ Scientist'}
{u'count': 2, u'_id': u'Trinity United Methodist Church'}
{u'count': 2, u'_id': u'First United Methodist Church'}
{u'count': 2, u'_id': u'Seventh Day Adventist Church'}
{u'count': 2, u'_id': u'Mountain View Chinese Christian Church'}
{u'count': 2, u'_id': u'Peninsula Bible Church'}
{u'count': 1, u'_id': u'Unity Missionary Baptist Church'}
{u'count': 1, u'_id': u'Cornerstone Community Church'}
{u'count': 1, u'_id': u"Saint Mark's Missionary Baptist Church"}
{u'count': 1, u'_id': u'Foothill Baptist Church'}
{u'count': 1, u'_id': u'Kannon Do Zen Meditation Center'}
{u'count': 1, u'_id': u'New Hope International'}
{u'count': 1, u'_id': u'Holy Korean Martyrs Catholic Church'}
{u'count': 1, u'_id': u'Ch\xf9a Gi\xe1c Minh'}
{u'count': 1, u'_id': u'Open Door Church of God in Christ'}
{u'count': 1, u'_id': u'Saint Athanasius Catholic Church'}
{u'count': 1, u'_id':

In [46]:
pipeline = [{"$match":{"amenity":{"$exists":1}, "amenity": "fuel", "name":{"$exists":1}}},
            {"$group":{"_id":"$name", "count":{"$sum":1}}},
            {"$sort":{"count":-1}}]
result = collection.aggregate(pipeline)
for r in result:
    print r

{u'count': 10, u'_id': u'Shell'}
{u'count': 8, u'_id': u'Valero'}
{u'count': 7, u'_id': u'Chevron'}
{u'count': 6, u'_id': u'76'}
{u'count': 6, u'_id': u'Arco'}
{u'count': 2, u'_id': u'valero'}
{u'count': 2, u'_id': u'ARCO'}
{u'count': 1, u'_id': u'World Oil'}
{u'count': 1, u'_id': u'Fair Oaks 76'}
{u'count': 1, u'_id': u'Willow Cove Gas'}
{u'count': 1, u'_id': u'Alliance Gasoline'}
{u'count': 1, u'_id': u'Westmore Chevron'}
{u'count': 1, u'_id': u'Shell Gas'}
{u'count': 1, u'_id': u'Conoco Phillips 76'}
{u'count': 1, u'_id': u'Union 76'}
{u'count': 1, u'_id': u'SAP Vehicles Network Demo - Gas Station'}
{u'count': 1, u'_id': u"Ranier's Service Station"}
{u'count': 1, u'_id': u'Alliance'}
{u'count': 1, u'_id': u'Rotten Robbie'}


In [47]:
pipeline = [{"$match":{"amenity":{"$exists":1}, "amenity": "fast_food", "name":{"$exists":1}}},
            {"$group":{"_id":"$name", "count":{"$sum":1}}},
            {"$sort":{"count":-1}}]
result = collection.aggregate(pipeline)
for r in result:
    print r

{u'count': 14, u'_id': u'Subway'}
{u'count': 7, u'_id': u"McDonald's"}
{u'count': 5, u'_id': u"Togo's"}
{u'count': 5, u'_id': u'Taco Bell'}
{u'count': 3, u'_id': u'Burger King'}
{u'count': 3, u'_id': u'Round Table Pizza'}
{u'count': 3, u'_id': u'KFC'}
{u'count': 2, u'_id': u'Jack In The Box'}
{u'count': 2, u'_id': u'In-N-Out Burger'}
{u'count': 2, u'_id': u'Jack in the Box'}
{u'count': 2, u'_id': u'Jamba Juice'}
{u'count': 2, u'_id': u"Wendy's"}
{u'count': 1, u'_id': u'Falafel Stop'}
{u'count': 1, u'_id': u'Peninsula Creamery'}
{u'count': 1, u'_id': u"Domino's Pizza"}
{u'count': 1, u'_id': u'Chick-Fil-A'}
{u'count': 1, u'_id': u"Carls' Jr."}
{u'count': 1, u'_id': u'Burger Town'}
{u'count': 1, u'_id': u'Tacobell-KFC'}
{u'count': 1, u'_id': u'Adamsons French Dipp'}
{u'count': 1, u'_id': u'El Pollo Loco'}
{u'count': 1, u'_id': u"Arby's"}
{u'count': 1, u'_id': u"In'n Out Burger"}
{u'count': 1, u'_id': u'zpizza'}
{u'count': 1, u'_id': u'Zume Pizza'}
{u'count': 1, u'_id': u'Cosmic Caf\xe9'}


In [52]:
pipeline = [{"$match":{"amenity":{"$exists":1}, "amenity": "hospital", "name":{"$exists":1}}},
            {"$group":{"_id":"$name", "count":{"$sum":1}}},
            {"$sort":{"count":-1}}]
result = collection.aggregate(pipeline)
for r in result:
    print r

{u'count': 1, u'_id': u'Grant Cuesta Sub Acute Rehabilitation Center'}
{u'count': 1, u'_id': u'VA Palo Alto Health Care System'}
{u'count': 1, u'_id': u'VA Medical Center Menlo Park'}
{u'count': 1, u'_id': u'El Camino Hospital'}
{u'count': 1, u'_id': u'PAMF Menlo Park Surgical Hospital'}
{u'count': 1, u'_id': u'Kaiser Permanente'}
{u'count': 1, u'_id': u'Kaiser Permanente Santa Clara Medical Center'}
{u'count': 1, u'_id': u'Camino Medical Group'}
{u'count': 1, u'_id': u'Palo Alto Medical Foundation'}
{u'count': 1, u'_id': u'Health Services'}


In [54]:
pipeline = [{"$match":{"amenity":{"$exists":1}, "amenity": "beauty", "name":{"$exists":1}}},
            {"$group":{"_id":"$name", "count":{"$sum":1}}},
            {"$sort":{"count":-1}}]
result = collection.aggregate(pipeline)
for r in result:
    print r

{u'count': 2, u'_id': u'Salon Elizabeth'}
{u'count': 1, u'_id': u'Salon 121'}


In [55]:
pipeline = [{"$match":{"amenity":{"$exists":1}, "amenity": "public_bookcase", "name":{"$exists":1}}},
            {"$group":{"_id":"$name", "count":{"$sum":1}}},
            {"$sort":{"count":-1}}]
result = collection.aggregate(pipeline)
for r in result:
    print r

{u'count': 3, u'_id': u'Little Free Library'}


In [56]:
pipeline = [{"$match":{"amenity":{"$exists":1}, "amenity": "school", "name":{"$exists":1}}},
            {"$group":{"_id":"$name", "count":{"$sum":1}}},
            {"$sort":{"count":-1}}]
result = collection.aggregate(pipeline)
for r in result:
    print r

{u'count': 2, u'_id': u'Lucille M Nixon Elementary School'}
{u'count': 2, u'_id': u'Palo Verde Elementary School'}
{u'count': 2, u'_id': u'Ohlone Elementary School'}
{u'count': 2, u'_id': u'Jane Lathrop Stanford Middle School'}
{u'count': 2, u'_id': u'Pinewood School'}
{u'count': 2, u'_id': u'Union Academy'}
{u'count': 2, u'_id': u'Stratford School'}
{u'count': 2, u'_id': u'Laurelwood Elementary School'}
{u'count': 2, u'_id': u'Athena Academy'}
{u'count': 2, u'_id': u'Kumon'}
{u'count': 1, u'_id': u'Bullis Charter School - South Campus'}
{u'count': 1, u'_id': u'Saint Elizabeth Ann Seton School'}
{u'count': 1, u'_id': u'Egan Junior High School'}
{u'count': 1, u'_id': u'Mountain View Los Altos Adult Education'}
{u'count': 1, u'_id': u"Mountain View-Los Altos Montessori Children's Centerter"}
{u'count': 1, u'_id': u'Terman Middle School'}
{u'count': 1, u'_id': u'Saint Simon Elemntary School'}
{u'count': 1, u'_id': u'Fairmeadow Elementary School'}
{u'count': 1, u'_id': u'Edison Brentwood A

In [57]:
pipeline = [{"$match":{"amenity":{"$exists":1}, "amenity": "parking", "name":{"$exists":1}}},
            {"$group":{"_id":"$name", "count":{"$sum":1}}},
            {"$sort":{"count":-1}}]
result = collection.aggregate(pipeline)
for r in result:
    print r

{u'count': 13, u'_id': u"Visitor's Parking"}
{u'count': 11, u'_id': u'Apartment Visitor Parking'}
{u'count': 3, u'_id': u'Customer Parking'}
{u'count': 3, u'_id': u'Employee Parking'}
{u'count': 2, u'_id': u'Lot 1'}
{u'count': 2, u'_id': u'Lot 4'}
{u'count': 2, u'_id': u'Sunnyvale Caltrain Station'}
{u'count': 2, u'_id': u'Lot 8'}
{u'count': 2, u'_id': u'Lot 6'}
{u'count': 2, u'_id': u'Lot 7'}
{u'count': 1, u'_id': u'Parking Lot A'}
{u'count': 1, u'_id': u'San Antonio Park and Ride'}
{u'count': 1, u'_id': u'SAP Vehicles Network Demo - Parking'}
{u'count': 1, u'_id': u'Civic Center Parking'}
{u'count': 1, u'_id': u'Car parking'}
{u'count': 1, u'_id': u'Car Parking'}
{u'count': 1, u'_id': u'Whisman'}
{u'count': 1, u'_id': u'San Antonio'}
{u'count': 1, u'_id': u'California Avenue'}
{u'count': 1, u'_id': u'Apple Mathilda 4'}
{u'count': 1, u'_id': u'Ted Thompson Parking Garage'}
{u'count': 1, u'_id': u'Lot 3'}
{u'count': 1, u'_id': u'parking lot'}
{u'count': 1, u'_id': u'Post office parking

In [58]:
pipeline = [{"$match":{"amenity":{"$exists":1}, "amenity": "car_wash", "name":{"$exists":1}}},
            {"$group":{"_id":"$name", "count":{"$sum":1}}},
            {"$sort":{"count":-1}}]
result = collection.aggregate(pipeline)
for r in result:
    print r

{u'count': 1, u'_id': u'SV Express'}
{u'count': 1, u'_id': u'Clear Water Car Wash'}
{u'count': 1, u'_id': u'Thrifty'}
{u'count': 1, u'_id': u'Lozano Brushless Car Wash'}
{u'count': 1, u'_id': u"Lozano's Car Wash"}
{u'count': 1, u'_id': u'Car Wash'}
{u'count': 1, u'_id': u'Shell'}
{u'count': 1, u'_id': u'Bubbles Hand Wash'}


In [59]:
pipeline = [{"$match":{"amenity":{"$exists":1}, "amenity": "post_box", "name":{"$exists":1}}},
            {"$group":{"_id":"$name", "count":{"$sum":1}}},
            {"$sort":{"count":-1}}]
result = collection.aggregate(pipeline)
for r in result:
    print r

{u'count': 1, u'_id': u'Mailbox'}


In [60]:
pipeline = [{"$match":{"amenity":{"$exists":1}, "amenity": "cafe", "name":{"$exists":1}}},
            {"$group":{"_id":"$name", "count":{"$sum":1}}},
            {"$sort":{"count":-1}}]
result = collection.aggregate(pipeline)
for r in result:
    print r

{u'count': 21, u'_id': u'Starbucks'}
{u'count': 3, u'_id': u'Starbucks Coffee'}
{u'count': 3, u'_id': u'Tea Era'}
{u'count': 2, u'_id': u'Philz Coffee'}
{u'count': 2, u'_id': u'Cloud Cafe'}
{u'count': 2, u'_id': u"Peet's Coffee"}
{u'count': 2, u'_id': u"Peet's Coffee & Tea"}
{u'count': 2, u'_id': u'Peets Coffee'}
{u'count': 1, u'_id': u'Happy Donuts'}
{u'count': 1, u'_id': u'Island Cafe'}
{u'count': 1, u'_id': u'Dana Street Roasting Company'}
{u'count': 1, u'_id': u'Maverick'}
{u'count': 1, u'_id': u'Badaal'}
{u'count': 1, u'_id': u'MiXiT Charleston'}
{u'count': 1, u'_id': u'Threads'}
{u'count': 1, u'_id': u'WAN'}
{u'count': 1, u'_id': u'California Mochi Mini'}
{u'count': 1, u'_id': u'Big Bites Vietnamese Eatery'}
{u'count': 1, u'_id': u'Taj Cafe'}
{u'count': 1, u'_id': u'Big Table'}
{u'count': 1, u'_id': u"Charlie's cafe"}
{u'count': 1, u'_id': u'The Lunch Box'}
{u'count': 1, u'_id': u'moma2'}
{u'count': 1, u'_id': u'Portal Cafe'}
{u'count': 1, u'_id': u'Backyard'}
{u'count': 1, u'_id

In [63]:
pipeline = [{"$match":{"amenity":{"$exists":1}, "amenity": "", "name":{"$exists":1}}},
            {"$group":{"_id":"$name", "count":{"$sum":1}}},
            {"$sort":{"count":-1}}]
result = collection.aggregate(pipeline)
for r in result:
    print r